In [202]:
import pandas as pd
import numpy as np

In [203]:
result = pd.read_csv('Travel_Survey_Compressed.csv', sep=',')
#result.head()

In [204]:
f = open('Travel_Survey_Data.csv', 'w')
with open('Travel_Survey_Compressed.csv', 'r') as r:
    for ln in r:
        ln_li = ln.replace("\n", "").replace("\ufeff","").split(',')
        for i in ln_li[0].split(";"):
            for j in ln_li[2].split(";"):
                new_ln = i + "," + ln_li[1] + "," + j.lstrip() + ","
                new_ln += ln.split(",", 3)[3] + "\n"
                new_ln = new_ln.replace("\n\n", "\n")
                #print(new_ln)
                f.write(new_ln.lstrip())
result_filtered = pd.read_csv('Travel_Survey_Data.csv', sep=',')

In [205]:
dest_list = result["Recommendation"].tolist()
dest_li_unique = list(dict.fromkeys(dest_list))
#print(dest_li_unique, len(dest_li_unique))

In [206]:
#favorite activity encoding
activity_list = result_filtered["Favorite Activity"].tolist()
activity_list = list(dict.fromkeys(activity_list))
#print(activity_list)

#reason to travel encoding
reason_list = result_filtered["Main Reason to Travel"].tolist()
reason_list = list(dict.fromkeys(reason_list))

#important factor encoding
factor_list = result_filtered["Most Important Factor for Trip Planning"].tolist()
factor_list = list(dict.fromkeys(factor_list))

#appealing destination encoding
appeal_list = result_filtered["Most Appealing Destination"].tolist()
appeal_list = list(dict.fromkeys(appeal_list))

# new csv
fw = open("Travel_Survey_Data_Encoded.csv", "w")
fw.write("Recommendation,Ideal Group Size,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,is_student,is_male,is_female,is_non_binary,")

en_text_list = ""
for act in reason_list + factor_list + activity_list + appeal_list:
    en_name = "is_" + act.replace(" ", "_").lower()
    en_text_list += en_name + ","
#print(en_text_list[:-1])
fw.write(en_text_list[:-1] + "\n")
#fw.close()

with open('Travel_Survey_Compressed.csv', 'r') as fr:
    for ln in fr:
        if "Recommendation" in ln:
            pass
        else:
            li = ln.lower().split(",")
            reason = li[0]
            factor = li[1]
            activity = li[2]
            appeal = li[3]
            gp_size = li[4]
            gender = li[5]
            student = li[6]
            op, con, ext, agr, neu = li[7:12]
            rec = li[12].replace("\n", "").title()
            
            row = rec + "," + gp_size + "," + op + "," + con + "," + ext + "," + agr + "," + neu + ","
            if student.strip().lower() == "yes":
                row += "1,"
            else:
                row += "0,"
            if gender.strip().lower() == "male":
                row += "1,"
            else:
                row += "0,"
            if gender.strip().lower() == "female":
                row += "1,"
            else:
                row += "0,"
            if gender.strip().lower() == "non_binary":
                row += "1,"
            else:
                row += "0,"
            
            #reason
            for i in reason_list:
                if i.lower() in reason:
                    row += "1,"
                else:
                    row += "0,"
                    
            #factor
            for i in factor_list:
                if i.lower() in factor:
                    row += "1,"
                else:
                    row += "0,"
                    
            #activity
            for i in activity_list:
                if i.lower() in activity:
                    row += "1,"
                else:
                    row += "0,"
                    
            #appeal
            for i in appeal_list:
                if i.lower() in appeal:
                    row += "1,"
                else:
                    row += "0,"
                    
            fw.write(row + "\n")
fw.close()

In [230]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(result_filtered, "Recommendation")
enc.categories_

[array(['ART and ENERGY', 'Art museums/galleries', 'Discover new cuisines',
        'FOOD', 'To climb', 'To explore architecture and history',
        'To go on an adventure',
        'To learn more about a language or a culture',
        'To look at and appreciate beautiful places', 'To meet new people',
        'To relax and escape work and responsibilities',
        'To see the wonders of nature',
        'To share travel stories and view with friends and family back home',
        'To spend quality time with family/friends', 'Work or business'],
       dtype=object),
 array(['Aesthetic of destination', 'Budget',
        'Overall quality of general experience', 'Proximity to nature',
        'Proximity to other destinations', 'Safety', 'Time',
        'Travel group size', 'Variety of activities at destination',
        'Variety of activities at destination but more specifically what climbing is in that area'],
       dtype=object),
 array(['Animals', 'Architecture', 'Art Gallery', '

In [207]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [208]:
train, test = train_test_split(result_filtered,test_size=0.15)

In [223]:
c = DecisionTreeClassifier()
features = ["Ideal Group Size","Openness","Conscientiousness","Extraversion","Agreeableness","Neuroticism"]

In [231]:
x_train = train[features]
y_train = train['Recommendation']

x_test = test[features]
y_test = test['Recommendation']
c.fit(enc, y_train)

AttributeError: 'OneHotEncoder' object has no attribute 'ndim'